In [2]:
import os
import codecs
import re
from time import sleep

In [3]:
# get google cloud
!pip install --upgrade google-cloud-translate
from google.cloud import translate

  Running setup.py bdist_wheel for googleapis-common-protos: started
  Running setup.py bdist_wheel for googleapis-common-protos: finished with status 'done'
  Stored in directory: C:\Users\User\AppData\Local\pip\Cache\wheels\da\6b\81\8573adcbe2aa2ecba92c341dfe19c5b5a733f4514297ba52b4
Successfully built googleapis-common-protos


In [4]:
# get nltk to calculate bleu score
!pip install --upgrade nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

Requirement already up-to-date: nltk in c:\users\user\anaconda3\lib\site-packages (3.4)


# Zulu

In [39]:
# load data
en_file = codecs.open('../clean/en_zu/enzu_parallel.test.en', 'r')
zu_file = codecs.open('../clean/en_zu/enzu_parallel.test.zu', 'r')

en_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in en_file.readlines()]
zu_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in zu_file.readlines()]

# translate
if not os.path.exists('./translated.zu'):
    trans_file = codecs.open('./translated.zu','w')
    
    # translate 20 sentences at a time
    translate_client = translate.Client()
    target = 'zu'
    for i in range(0, len(en_sents) // 20):
        translations = translate_client.translate(
            en_sents[i*20:(i+1)*20],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')

        print("Translated %d sentences" %((i+1)*20))
        sleep(1.5)
        
    trans_file.close()
    
# calculate bleu score
trans_file = codecs.open('./translated.zu', 'r')
trans_sents = [line.rstrip('\n').split(' ') for line in trans_file.readlines()]
num_trans = len(trans_sents)

zu_refs = [[re.sub(r'\s+([?.!,"])', r'\1', line).split(' ')] for line in zu_sents]

bleu_score = corpus_bleu(zu_refs[:num_trans], trans_sents)

print(bleu_score*100)

trans_file.close()
en_file.close()
zu_file.close()

7.546014989687105


In [15]:
num_trans

2880

In [37]:
n = 65

print(en_sents[n])
print(zu_refs[n])
print(trans_sents[n])
print(sentence_bleu(zu_refs[n], trans_sents[n])*100)


CS .0171 Annual contract for the supply , delivery , installation , testing and commissioning of fibre network monitoring system for 12/24/36 month period Closing date: 18 May 2007
[['CS.0171', 'Inkontileka', 'yonyaka', 'yokuthengisela,', 'ukudiliva,', 'ukufaka,', 'ukuhlola', 'nokubeka', 'esimweni', 'sokusebenza', 'i-fibre', 'network', 'monitoring', 'system', 'izinyanga', 'ezingu', '12/24/36', 'Kuvalwa:', '18', 'Meyi', '2007']]
['CS', '.0171', 'Inkontileka', 'yonyaka', 'yokuhlinzeka,', 'ukulethwa,', 'ukufakwa,', 'ukuhlolwa', 'nokuthunyelwa', 'kohlelo', 'lokuqapha', 'kwenethiwekhi', 'ye-fibre', 'ngo', '12/24/36', 'inyanga', 'inyanga', 'yokuvalwa:', '18', 'Meyi', '2007']
2.6616657200018398e-76


In [107]:
zu_sents[0]

'Imibhalo yamathenda itholakala eThekwini Electricity , 1 Jelf Taylor Crescent , Durban ngo 08:00 kuya ku 15:00 , ngeMisombuluko kuya'